In [ ]:
# importing dependencies
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
from matplotlib import pyplot as plt


In [ ]:
# reading in data file
df = pd.read_csv('../Resources/application_train.csv')


In [ ]:
# viewing the dataframe
df

In [ ]:
#Check for and remove outliers
from scipy import stats
num_types = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df[(np.abs(stats.zscore(df.select_dtypes(include=num_types), nan_policy='omit')) < 3).all(axis=1)]
df

#### No outliers detected in the data.

In [ ]:
# removing columns that are less relevant to analysis
df2 = df.drop(['FLAG_MOBIL', "FLAG_EMP_PHONE","FLAG_WORK_PHONE", "FLAG_CONT_MOBILE", "FLAG_EMAIL", "REG_REGION_NOT_LIVE_REGION", "REG_REGION_NOT_WORK_REGION", "LIVE_REGION_NOT_WORK_REGION", "REG_CITY_NOT_LIVE_CITY", "REG_CITY_NOT_WORK_CITY", "LIVE_CITY_NOT_WORK_CITY", "FLAG_DOCUMENT_2", "FLAG_DOCUMENT_3", 
               "FLAG_DOCUMENT_4", "FLAG_DOCUMENT_5", "FLAG_DOCUMENT_6", "FLAG_DOCUMENT_7", "FLAG_DOCUMENT_8", "FLAG_DOCUMENT_9", "FLAG_DOCUMENT_10", "FLAG_DOCUMENT_11", "FLAG_DOCUMENT_12", "FLAG_DOCUMENT_13", "FLAG_DOCUMENT_14", "FLAG_DOCUMENT_15", "FLAG_DOCUMENT_16", "FLAG_DOCUMENT_17", "FLAG_DOCUMENT_18", "FLAG_DOCUMENT_19",
               "FLAG_DOCUMENT_21", "_id", "SK_ID_CURR"], axis=1)

df2

In [ ]:
# displaying the data types
pd.set_option('display.max_rows', None)
display(df2.dtypes)

In [ ]:
# counting the number of nulls in each column
new_null= print(df2.isnull().sum())

In [ ]:
# removing null values from dataframe
pd.set_option('display.max_rows', 10)
no_nulls = df2.dropna(axis=0)

no_nulls

In [ ]:
## Data Preprocessing
X = df2.drop('TARGET',1)
y = df2['TARGET']


In [ ]:
# one-hot encoding for categorical variables
categorical_variables = ['EMERGENCYSTATE_MODE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE',
                         'NAME_CONTRACT_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
                         'NAME_INCOME_TYPE', 'NAME_TYPE_SUITE', 'OCCUPATION_TYPE', 'ORGANIZATION_TYPE', 
                         'WALLSMATERIAL_MODE', 'WEEKDAY_APPR_PROCESS_START', 'CODE_GENDER']
                
for variable in categorical_variables:
  discarded= pd.get_dummies(X[variable], prefix=variable)
  X= pd.concat([X, discarded], axis = 1)
  X.drop([variable], axis=1, inplace=True)

X

In [ ]:
# displaying the data types in X after one-hot encoding
pd.set_option('display.max_rows', None)
display(X.dtypes)

In [ ]:
# viewing the correlations of the X dataframe
#X.corr()

In [ ]:
# viewing y dataframe
pd.set_option('display.max_rows', 10)
y

In [ ]:
#Use SimpleImputer to replace missing values
from sklearn.impute import SimpleImputer
simple_imp= SimpleImputer( strategy='most_frequent')
simple_imp.fit(X)
imputed_train_df = simple_imp.transform(X)

In [ ]:
imputed_train_df = pd.DataFrame(imputed_train_df)
imputed_train_df

In [ ]:
imputed_train_df.isnull().values.any()

In [ ]:
# saving results to csv file
simple_imputationX = imputed_train_df.to_csv("simple_imputationX.csv")
y_application = y.to_csv("y.csv")

In [ ]:
# reading in data for random forest classifier
X = pd.read_csv('simple_imputationX.csv')
y = pd.read_csv('y.csv')

Dimensionality Reduction

In [ ]:
# perform train/test split, scaling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
#Fit Random Forest model for dimensionality reduction
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train, y_train)
print(f'Training Score: {clf.score(X_train, y_train)}')
print(f'Testing Score: {clf.score(X_test, y_test)}')

In [ ]:
# plotting the features to determine their importance in the model
features = sorted(zip(X.columns, clf.feature_importances_), key = lambda x: x[1])
cols = [f[0] for f in features]
width = [f[1] for f in features]
fig, ax = plt.subplots()
fig.set_size_inches(10,200)
plt.margins(y=0.001)
ax.barh(y=cols, width=width)
plt.show()

In [ ]:
# Import and fit SelectFromModel
from sklearn.feature_selection import SelectFromModel
sel = SelectFromModel(clf)
sel.fit(X_train_scaled, y_train)

In [ ]:
# Use selector to transform X
X_selected_train, X_selected_test, y_train, y_test = train_test_split(sel.transform(X), y, random_state=1)
scaler = StandardScaler().fit(X_selected_train)
X_selected_train_scaled = scaler.transform(X_selected_train)
X_selected_test_scaled = scaler.transform(X_selected_test)

Supervised Learning Model

In [ ]:
# import and creat logistic regression classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier


In [ ]:
# fit the model to reduced, scaled datset
classifier.fit(X_selected_train_scaled, y_train)

In [ ]:
# fit the model score
print(f"Training Data Score: {classifier.score(X_selected_train_scaled, y_train)}")
print(f"Testing Score: {classifier.score(X_selected_test_scaled, y_test)}")

In [ ]:
# generate confusion matrix for logistic regression
y_true = y_test
y_pred = classifier.predict(X_selected_test_scaled)
cm = confusion_matrix(y_true, y_pred)
cm

In [ ]:
# generate classification report for logistic regression
print(classification_report(y_true, y_pred)

In [ ]:
# import linear svc classifier
from sklearn.svm import LinearSVC

In [ ]:
# attempt lineaer support vector classification
lin_svc_classifier = LinearSVC(randwom_state=1). fit(X_selected_train_scale, y_train)
y_pred = lin_svc_classifier.predict(X_selected_test_scaled)
print(classificatoin_report(y_Test, y_pred, target_Names-target_names))
print(f'Training Score: {lin_svc_classifier.score(X_selected_train_scaled, y_train)}')
print(f'Training Score: {lin_svc_classifier.score(X_selected_train_scaled, y_train)}')


In [ ]:
#import K-nearest neighbor classifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# perform KNN classification, to find the most optimal value of k
# Loop through different k values to find which has the highest accuracy.
# Note: We use only odd numbers because we don't want any ties.
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    train_score = knn.score(X_train_scaled, y_train)
    test_score = knn.score(X_test_scaled, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
    
plt.plot(range(1, 20, 2), train_scores, marker='o')
plt.plot(range(1, 20, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy score")
plt.show()

In [ ]:
# using k value selection, perform KNN Classification, and print report to compare


In [ ]:
# import an extremely randomw trees classifier
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
# fit RandomTressClassifier, and print classification report to compare
ext_classifier= ExtraTreesClassifier(raond_state=1).fit(X_Selected_train_selected, y_train)

In [ ]:
# Note that k: xx seems to be the best choice for this dataset
knn = KNeighborsClassifier(n_neighbors= )
knn.fit(X_train_scaled, y_train)
print('k=  Test Acc: %.3f' % knn.score(X_test_scaled, y_test))